In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE66843"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE66843"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE66843.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE66843.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE66843.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A cell-based model unravels drivers for hepatocarcinogenesis and targets for clinical chemoprevention"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['time post infection: Day 3 post infection', 'time post infection: Day 7 post infection', 'time post infection: Day 10 post infection'], 1: ['infection: Mock infection (control)', 'infection: HCV Jc1 infection'], 2: ['cell line: Huh7.5.1']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Dict, Any, Callable

# Determine Gene Expression Availability
# This dataset appears to be HCV infection in cell lines (Huh7.5.1), which likely contains gene expression data
is_gene_available = True

# Analyze trait, age, and gender availability
# From the sample characteristics, we have:
# - trait appears to be infection status (HCV vs Mock)
# - no age data (these are cell lines)
# - no gender data (these are cell lines)

trait_row = 1  # The row containing infection status
age_row = None  # No age data for cell lines
gender_row = None  # No gender data for cell lines

# Define conversion functions
def convert_trait(value):
    """Convert infection status to binary: HCV=1, Mock=0"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'mock' in value.lower() or 'control' in value.lower():
        return 0
    elif 'hcv' in value.lower() or 'jc1' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion"""
    # Not applicable for this dataset
    return None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    # Not applicable for this dataset
    return None

# Determine trait data availability
is_trait_available = trait_row is not None

# Save metadata with initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Load clinical data from a previous step
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 46116 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers to determine if they need mapping
# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# These are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE66843
Line 6: !Series_title = A cell-based model unravels drivers for hepatocarcinogenesis and targets for clinical chemoprevention
Line 7: !Series_geo_accession = GSE66843
Line 8: !Series_status = Public on Jun 28 2021
Line 9: !Series_submission_date = Mar 12 2015
Line 10: !Series_last_update_date = Jun 29 2022
Line 11: !Series_pubmed_id = 34535664
Line 12: !Series_summary = This SuperSeries is composed of the SubSeries listed below.
Line 13: !Series_overall_design = Refer to individual Series
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_type = Expression profiling by high throughput sequencing
Line 16: !Series_sample_id = GSM1633141
Line 17: !Series_sample_id = GSM1633142
Line


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the column names for probe IDs and gene symbols from the annotation preview
# From the annotation preview, we can see:
# - 'ID' contains the gene identifiers (ILMN_*) which match the gene expression data
# - 'Symbol' contains the gene symbols we want to map to

# 2. Get a gene mapping dataframe from the annotation
print("Creating gene mapping from annotation data...")
try:
    # Get gene mapping by selecting ID and Symbol columns from the gene annotation
    gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
    print(f"Created gene mapping with {len(gene_mapping)} rows")
    print("Sample of gene mapping:")
    print(gene_mapping.head())
    
    # Check proportion of IDs that have gene symbols
    valid_symbols = gene_mapping['Gene'].notnull().sum()
    total_rows = len(gene_mapping)
    print(f"Proportion of probes with valid gene symbols: {valid_symbols}/{total_rows} ({valid_symbols/total_rows:.1%})")
    
    # 3. Convert probe-level data to gene expression data
    print("\nConverting probe-level measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Created gene expression data with {len(gene_data)} genes")
    print("First few genes:")
    print(gene_data.index[:10])
    
    # Check gene_data dimensions
    print(f"Gene expression matrix shape: {gene_data.shape}")
    
    # Preview a small sample of the gene data
    print("\nPreview of gene expression data:")
    sample_genes = gene_data.iloc[:5, :5]
    print(sample_genes)
    
    # Save the gene data
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save the gene expression data
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping process: {e}")
    is_gene_available = False


Creating gene mapping from annotation data...
Created gene mapping with 44837 rows
Sample of gene mapping:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Proportion of probes with valid gene symbols: 44837/44837 (100.0%)

Converting probe-level measurements to gene expression data...
Created gene expression data with 21125 genes
First few genes:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')
Gene expression matrix shape: (21125, 17)

Preview of gene expression data:
        GSM1633236   GSM1633237   GSM1633238   GSM1633239   GSM1633240
Gene                                                                  
A1BG    175.902881   175.467437   185.163824   175.411354   181.091704
A1CF   2058.146767  1601.

Gene expression data saved to ../../output/preprocess/Hepatitis/gene_data/GSE66843.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (21125, 17)
Gene data shape after normalization: (19956, 17)


Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE66843.csv
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE66843.csv
Linked data shape: (17, 19957)

Handling missing values...


After missing value handling, linked data shape: (17, 19957)

Evaluating feature bias...
For the feature 'Hepatitis', the least common label is '0.0' with 8 occurrences. This represents 47.06% of the dataset.
The distribution of the feature 'Hepatitis' in this dataset is fine.

Trait bias evaluation result: False

Dataset usability: True


Linked data saved to ../../output/preprocess/Hepatitis/GSE66843.csv
